# ベイズ勉強会 Part 6 カテゴリ分布

> カテゴリ分布のベイズ推論 [カバー画像元](https://ja.m.wikipedia.org/wiki/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB:Sixsided_Dice_inJapan.jpg)

- toc: true 
- badges: true
- comments: true
- categories: [bayes]
- image: images/dag1.png

ベイズ勉強会資料は『ベイズ推論による機械学習入門』{% fn 1 %}を元に、途中式計算をできるだけ省略せずに行ったものです。

## カテゴリ分布

カテゴリ分布は次のような確率質量関数を分布関数にもつ確率分布である。

$$
Cat({\bf s}|{\bf \pi}) = \Pi_{k=1}^{K} \pi_k^{s_k}
$$

${\bf s}$はK次元ベクトルで、該当カテゴリのみを1,それ以外を0で表したものである(1 of K 表現)。例えば、サイコロの目を確率変数とすれば$s = 5$と書く代わりに、${\bf s} = (0,0,0,0,1,0)^\mathrm{T}$と書く。パラメータ${\bf \pi} = (\pi_1,\dots,\pi_K)^\mathrm{T}$は各カテゴリに配される確率を表したものであり、$\pi_k \in (0,1)$かつ$\Sigma_{k=1}^{K} \pi_k = 1$を満たす。

対数をとると

$$
\ln Cat({\bf s}|{\bf \pi}) = \Sigma_{k=1}^{K} s_k \ln \pi_k
$$

となる。

## ディリクレ分布

尤度関数にカテゴリ分布をとった場合、パラメータ${\bf \pi}$の事前分布は、$\pi_k \in (0,1)$かつ$\Sigma_{k=1}^{K} \pi_k = 1$を満たすK次元ベクトルを出力する必要がある。このような分布として、ディリクレ分布がある。ディリクレ分布の分布関数は次のようになる。

$$
Dir({\bf \pi}|{\bf \alpha}) = C_D ({\bf \alpha}) \Pi_{k=1}^{K} \pi_k^{\alpha_k - 1}
$$

ディリクレ分布のパラメータ${\bf \alpha} = (\alpha_1, \dots, \alpha_K)^\mathrm{T}$の要素$\alpha_k$は正の実数である。正規化係数は

$$
C_D ({\bf \alpha}) = \frac{\Gamma (\Sigma_{k=1}^{K} \alpha_k)}{\Pi_{k=1}^{K} \Gamma (\alpha_k)}
$$

である。

対数をとると

$$
\ln Dir({\bf \pi}|{\bf \alpha}) = \Sigma_{k=1}^{K} (\alpha_k - 1) \ln \pi_k + \ln C_D ({\bf \alpha})
$$

となる。

## モデル構築

サイコロの各目が出る確率をベイズ推論で求めてみる。そのためのモデルは次のように書ける。

$$
\begin{eqnarray}
    p({\bf S}, {\bf \pi}) &=& p({\bf S}|{\bf \pi})p({\bf \pi}) \\
    ただし　p({\bf S}|{\bf \pi}) &=& \Pi_{n=1}^{N} Cat({\bf s}_n|{\bf \pi}) \\
    p({\bf \pi}) &=& Dir({\bf \pi}|{\bf \alpha})
\end{eqnarray}
$$

観測されたデータを${\bf S} = \{ {\bf s}_1, \dots, {\bf s}_N \}$とする。事前分布のパラメータ${\bf \alpha}$はハイパーパラメータとなる。

## 事後分布の推論

事後分布$p({\bf \pi}|{\bf S})$は同時分布を観測されたデータ${\bf S}$の確率分布で割れば求まる。ただし${\bf S}$は${\bf \pi}$を含んでおらず対数をとることにより定数項として扱える。

$$
\begin{eqnarray}
    \ln p({\bf \pi}|{\bf S}) &=& \Sigma_{n=1}^{N} \ln Cat({\bf s}_n | {\bf \pi}) + \ln Dir({\bf \pi}|{\bf \alpha}) + const. \\
    &=& \Sigma_{n=1}^{N} \Sigma_{k=1}^{K} s_{n,k} \ln \pi_k + \Sigma_{k=1}^{K} (\alpha_k - 1) \ln \pi_k + \ln C_D ({\bf \alpha}) + const. \\
    &=& \Sigma_{k=1}^{K} (\Sigma_{n=1}^{N} s_{n,k} + \alpha_k - 1) \ln \pi_k + const.
\end{eqnarray}
$$

これはディリクレ分布の対数をとった形である。よって事後分布は次のように表せる。

$$
\begin{eqnarray}
    p({\bf \pi}|{\bf S}) &=& Dir({\bf \pi} | \hat{\bf \alpha}) \\
    ただし　\hat{\bf \alpha}_k &=& \Sigma_{n=1}^{N} s_{n,k} + \alpha_k　\mathrm{for}　k = 1,\dots,K
\end{eqnarray}
$$

## 予測分布の導出

パラメータ${\bf \pi}$の事前分布と事後分布の形状が同じディリクレ分布となるので、事前分布を用いて事前予測分布を求めた後、学習済みのハイパーパラメータを代入して学習済み予測分布を求める。未観測のカテゴリ変数${\bf s}_*$(1 of K表現)は観測${\bf S}$と同じ分布から独立に生成されるとして、予測分布$p({\bf s}_*)$は次のように求まる。

$$
\begin{eqnarray}
    p({\bf s}_*) &=& \int p({\bf s}_*|{\bf \pi}) p({\bf \pi}) d {\bf \pi} \\
    &=& \int Cat({\bf s}_* | {\bf \pi}) Dir({\bf \pi} | {\bf \alpha}) d {\bf \pi} \\
    &=& C_D ({\bf \alpha}) \int \Pi_{k=1}^{K} \pi_k^{s_{*,k} } \pi_k^{\alpha_k -1} d {\bf \pi} \\
    &=& C_D ({\bf \alpha}) \int \Pi_{k=1}^{K} \pi_k^{s_{*,k} + \alpha_k - 1 } d {\bf \pi} \\
    &=& \frac{C_D ({\bf \alpha}) }{C_D ((s_{*,k} + \alpha_k)_{k=1}^{K})}　(ディリクレ分布の正規化係数を省いた形を積分すれば、正規化係数の逆数になるはず) \\
    &=& \frac{\Gamma(\Sigma_{k=1}^{K} \alpha_k) \Pi_{k=1}^{K} \Gamma (s_{*,k} + \alpha_k)}{\Pi_{k=1}^{K} \Gamma (\alpha_k) \Gamma(\Sigma_{k=1}^{K} (s_{*,k}+\alpha_k))}
\end{eqnarray}
$$

${\bf s}_*$の実現値を代入することで簡単にできる。ある$k'$に対して$s_{*,k'}=1$となる場合を考えると、

$$
\begin{eqnarray}
    p(s_{*,k'} = 1) &=& \frac{\Gamma(\Sigma_{k=1}^{K} \alpha_k) \Gamma (1 + \alpha_{k'})}{\Gamma (\alpha_{k'}) \Gamma(1 + \Sigma_{k=1}^{K} \alpha_k)} \\
    &=& \frac{\alpha_{k'} }{\Sigma_{k=1}^{K} \alpha_k}　(ガンマ関数と階乗の関係を利用)
\end{eqnarray}
$$

これはカテゴリ分布にまとめることができて、

$$
\begin{eqnarray}
    p({\bf s}_*) = Cat({\bf s}_* | \left( \frac{\alpha_k}{\Sigma_{i=1}^{K} \alpha_i} \right)_{k=1}^{K} )
\end{eqnarray}
$$

何をしているかというと、ハイパーパラメータ${\bf \alpha}$の各成分を正規化しただけである。

これを$\hat{\bf \alpha}$に置き換えれば学習後の予測分布が求まる。

{{ '[須山敦志. 杉山将. ベイズ推論による機械学習入門. 講談社, 2017.](https://www.kspub.co.jp/book/detail/1538320.html)' | fndetail: 1 }}